# Глубокое обучение на TensorFlow?
**Мовенко Константин**

Вы уже написали много кода во второй части заданий, чтобы реализовать весь набор функций необходимых для создания различных нейронных сетей. Dropout, блочная нормализация и двумерные свертки являются рабочими лошадками глубинного обучения в компьютерном зрении. Вы также приложили много усилий, чтобы сделать ваш код эффективным и векторизованным.

Однако в данном блокноте мы не станем использовать ваш код, а перейдем к одной из  популярных платформ глубокого обучения - TensorFlow

## Что такое TensorFlow?
TensorFlow - это система вычислений над тензорными объектами с использованием вычислительных графов и встроенной поддержкой выполнения обратного распространения. Тензоры представляют собой n-мерные массивы, аналогичные numpy ndarray.

## Зачем изучать TensorFlow?
Наш код теперь сможет исполняться на графических процессорах! В этом случае обучение будет проходить гораздо быстрее.
Мы хотим, чтобы Вы были готовы использовать один из развитых фреймворков для своих проектов, чтобы Вы могли проводить эксперименты эффективнее, чем если бы Вы писали каждую функцию вручную.
Мы хотим, чтобы Вы стояли на плечах гигантов! TensorFlow и PyTorch - отличные фреймворки, которые сделают вашу жизнь намного проще.
Мы хотим, чтобы Вы ознакомились с подходом к кодированию глубинного обучения, который применяется в академических кругах или в промышленности.

## Как изучать TensorFlow?

В интернете вы найдете много руководств по TensorFlow.

Этот блокнот также послужит Вам руководством по разработке и обучению моделей в TensorFlow. Просмотрите имеющиеся в  этом блокноте некоторые ссылки на полезные руководства, если вы хотите узнать больше или если Вам требуются дополнительные разъяснения.

**ПРИМЕЧАНИЕ. Этот блокнот предназначен для изучения последней версии Tensorflow 2.0. Большинство примеров в Интернете по-прежнему относятся к версии 1.x, поэтому будьте осторожны, чтобы не перепутать их при поиске документации**.

## Установка Tensorflow 2.X

 Tensorflow 2.X  проще в использовании и более интуитивен, чем TF 1.x. Пожалуйста, убедитесь, что он установлен, прежде чем переходить к этому блокноту! Вот несколько шагов по установке (некоторые шаги могут быть пропущены, если вы из делали ранее):

1. Установите последнюю версию Anaconda на вашем компьютере.
2. Создайте новую среду conda с Python 3.6 или 3.7: `conda create --name tf2_env python=3.6`
   Здесь `tf2_env` имя среды, которое мы выбрали сами.
3. Активируйте созданную среду командой: `conda activate tf2_env`
4. Установите  TF версии 2.0 или выше командой: `conda install tensorflow`
5. Проверьте установку TF, выполнив код:

   `import tensorflow as tf
    print(tf.__version__)`
    
 В результате будет напечатан номер установленной версии Tensorflow. Данные задания были проверены в версии 2.1.0 


# Содержание

Этот блокнот состоит из 5 частей. Мы будем рассматривать  TensorFlow на трех разных уровнях абстракции , что должно способствовать лучшему пониманию.

1. Часть I, Подготовка: загрузка множества данных CIFAR-10.
2. Часть II, Базовый TensorFlow (**уровень 1**): непосредственно используются низкоуровневые возможности TensorFlow.
3. Часть III, API объекта tf.keras.Model (**уровень 2**): используется  `tf.keras.Model` для определения произвольной архитектуры нейронной сети.
4.  Часть IV, Последовательный и функциональные API Keras (**уровень 3**): используется объект `tf.keras.Sequential` для определения простой линейной структуры сети прямого распространения и функциональные вызовы слоев сети при построении более сложных взаимосвязей. 
5. Часть V, Открытая задача классификации изображений  CIFAR-10: выполняется построение сети, которая способна обеспечить  точность классификации более 70% для базы изображений CIFAR-10. Вы можете экспериментировать с любым слоем, оптимизатором, гиперпараметрами или другими дополнительными свойствами.

Сравнительная таблица свойств программных интерфейсов (API):

| API           | Гибкость    | Простота    |
|---------------|-------------|-------------|
| Базовый       | Высокая     | Низкая      |
| `tf.keras.Model`     | Высокая| Средняя   |
| `tf.keras.Sequential`| Низкая | Высокая   |


# Часть I: Подготовительная

Сначала загрузим набор данных CIFAR-10. Это может занять несколько минут для загрузки при первом запуске, но после этого файлы должны быть кэшированы на диске, а загрузка должна быть быстрее.

В предыдущих частях задания мы использовали специфический код для загрузки и чтения набора данных CIFAR-10; однако пакет `tf.keras.datasets` в TensorFlow предоставляет предустановленные утилиты для загрузки многих распространенных наборов данных.

Для целей задания мы по-прежнему будем писать собственный код для предварительной обработки данных и итерации на данных по мини-блокам. Модуль `tf.data` в TensorFlow предоставляет необходимые инструменты для автоматизации этого процесса, однако работа с этим модулем  выходит за рамки заданий, рассматриваемых в этом блокноте. Тем не менее, использование `tf.data` может быть намного эффективнее, чем простой подход, рассматриваемый ниже. Поэтому самостоятельно познакомьтесь с модулем  `tf.data` и используйте его в своих проектах.

In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Функция извлекает набор данных CIFAR-10 из Интернета и выполняет предварительную
    обработку данных. 
    Такие же шаги, что мы использовали ранее в блокноте SVM, но теперь они собраны в одной функции.
      
    """
    # Загрузка базы CIFAR-10 и приведение данных к необходимым типам и формам
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Создание подмножеств данных
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Нормализация данных: вычитание средних значений пикселей и деление на стандартное отклонение
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Если при SSL загрузке  возникают ошибки, связанные с самозаверяющими сертификатами,
# возможно, ваша версия Python была недавно установлена на данном компьютере.
# См .: https://github.com/tensorflow/tensorflow/issues/10779
# Чтобы исправить, запустите команду: /Applications/Python\ 3.7 /Install\ Certificates.command
# ... заменив пути при необходимости.

# Вызоваем вышеопределенную функцию, чтобы загрузить данные.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Форма обучающих данных: ', X_train.shape)
print('Форма меток обучающих данных: ', y_train.shape, y_train.dtype)
print('Форма валидационных данных: ', X_val.shape)
print('Форма меток валидационных данных: ', y_val.shape)
print('Форма тестовых данных: ', X_test.shape)
print('Форма меток тестовых данных: ', y_test.shape)

Форма обучающих данных:  (49000, 32, 32, 3)
Форма меток обучающих данных:  (49000,) int32
Форма валидационных данных:  (1000, 32, 32, 3)
Форма меток валидационных данных:  (1000,)
Форма тестовых данных:  (10000, 32, 32, 3)
Форма меток тестовых данных:  (10000,)


### Предварительная подготовка: объект Dataset

Для удобства мы определим упрощенный класс `Dataset`, который позволит нам перебирать данные и метки. Это не самый гибкий или эффективный способ перебора данных, но он будет служить нашим целям.

In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Создание объекта Dataset для итераций по набору данных X и  меткам y
        
         Входы:
         - X: numpy массив данных любой формы
         - y: numpy массив меток любой формы, но с y.shape [0] == X.shape [0]
         - batch_size: целое число, указывающее количество элементов миниблока
         - shuffle: (необязательный) Логическое значение, указывает следует ли 
           перемешивать данные на каждой эпохе
        """
                
        assert X.shape[0] == y.shape[0], 'Проверка совпадения количества данных и меток'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))

# создаем экземпляры класса Dataset для работы с разными подмножествами данных
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# Мы можем выполнять итерации так:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


Вы можете опционально использовать GPU, установив флаг **USE_GPU в True** ниже. Для этого задания не обязательно использовать GPU

In [5]:
# Задание некоторых глобальных переменных
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Константа,определяющая через сколько итераций будет выполняться печать сообщений при обучении моделей
print_every = 100

print('Using device: ', device)

Using device:  /cpu:0


# Часть II: Базовый интерфейс TensorFlow

TensorFlow поставляется с различными высокоуровневыми API, что делает его очень удобным для определения и обучения нейронных сетей; мы рассмотрим некоторые из этих конструкций в части III и части IV этого блокнота. В этом разделе мы начнем с построения модели с базовыми конструкциями TensorFlow, чтобы далее помочь вам лучше разобраться с тем, что происходит под капотом высокоуровневых API.

**«Базовый Tensorflow» важен для понимания строительных блоков TensorFlow. Значительная его часть включает в себя концепции из TensorFlow 1.x.** Поэтому мы будем использовать старые модули, например `tf.Variable`.

Прочитайте ниже и поймите различия между устаревшей версией TF 1.x и новой TF 2.0.

### Основная философия  TensorFlow 1.x
TensorFlow 1.x, в первую очередь, - это фреймворк для  работы со **статическими вычислительными графами**. Ребрами  вычислительного графа являются тензоры, которые хранят n-мерные массивы; узлы  графа представляют собой функции, которые применяются к тензорам, когда выполняются вычисления в соответствии с вычислительным графом.

Это означает, что типичная программа с использованием TensorFlow 1.x выполняется в два этапа:

1. **Создание вычислительного графа, который описывает вычисления, подлежащие выполнению.** Этот этап фактически не выполняет никаких вычислений; он просто создает символическое представление ваших вычислений. Этот этап обычно определяет один или несколько объектов типа `placeholder`, которые представляют входные данные вычислительного графа.
2. **Многократное исполнение вычислительного графа.** Каждый раз, когда граф исполняется, вы указываете, какие части графа вы хотите вычислить, и передаёте словарь `feed_dict`, который поставляет конкретные значения любому объекту ` placeholder `на вход графа.

### Новая парадигма в Tensorflow 2.0
В Tensorflow 2.0, мы можем  использовать Python-подобные функциональные формы, которые более похожи по духу на PyTorch и  операции  Numpy библиотеки. Вместо двухэтапной парадигмы работы со статическим графом вычислений, Tensorflow 2.0 использует динамический граф, что облегчает (помимо прочего) отладку кода TF 2.0.
Более подробно об этом можно почитатать здесь https://www.tensorflow.org/guide/eager.

В TF 2.0 не используются модули TF 1.x, такие как : `tf.Session`,` tf.run`, `placeholder`,` feed_dict`. Чтобы получить детальную информацию о различиях между двумя версиями и о том, как выполнять преобразование между ними, ознакомьтесь с официальным руководством по миграции: https://www.tensorflow.org/alpha/guide/migration_guide.

Ниже, в соответствующих частях этого блокнота, мы будем рассмотривать этот новый, более простой подход.

### Разминка с TensorFlow: функция flatten

Определим простую функцию «уплощения данных» `flatten`, которая реформатирует данные изображений для использования в полносвязанной нейронной сети.

В TensorFlow данные для сверточных карт признаков  обычно хранятся в тензоре формы N x H x W x C, где:

- N - количество примеров данных (размер мини-блока);
- H - высота карты; 
- W - ширина карты; 
- C - количество каналов карты.

Это обычный способ представления данных для двумерной свертки, которая учитывает пространственные отношения между признаками изображений. Однако, когда мы используем полносвязанные слои нейронов для обработки изображений, то требуется, чтобы каждое  изображение представлялось вектором. Поэтому необходимо реформатировать изображение размером «H x W x C» в один "длинный" вектор. 

Обратите внимание,  что вызов метода `tf.reshape` в ячейке ниже использует аргумент `(N, -1)`, что означает, что он оставит первое измерение данных равным N, а затем  автоматически определит , каким должно быть  второе измерение выходных данных.  

**ПРИМЕЧАНИЕ**: TensorFlow и PyTorch различаются своими представлениями тензоров по умолчанию; TensorFlow использует представление N x H x W x C, а PyTorch использует N x C x H x W.


In [6]:
def flatten(x):
    """    
    Входные данные:
     - Тензор формы (N, D1, ..., DM)
    
     Выход:
     Тензор формы (N, D1 * ... * DM)
    """
    
    N = tf.shape(x)[0]
    return tf.reshape(x, (N, -1))

In [7]:
def test_flatten():
    # Создание массива х с использованием функций numpy
    x_np = np.arange(24).reshape((2, 3, 4))
    print('x_np:\n', x_np, '\n')
    # вычисление выходного массива
    x_flat_np = flatten(x_np)
    print('x_flat_np:\n', x_flat_np, '\n')

test_flatten()

x_np:
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]] 

x_flat_np:
 tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [12 13 14 15 16 17 18 19 20 21 22 23]], shape=(2, 12), dtype=int32) 



### Базовый  TensorFlow: Двухслойная сеть
Теперь мы реализуем нашу первую нейронную сеть на TensorFlow: двухслойную сеть с двумя полносвязными слоями без смещений и с ReLU нелинейностью. Пока будем использовать только низкоуровневые операторы TensorFlow для определения сети; позже мы увидим, как использовать абстракции более высокого уровня, предоставляемые `tf.keras`, чтобы упростить процесс.

Определим функцию прямого распространения  `two_layer_fc`; она будет принимать тензоры входа и весов сети и возвращать тензор оценки предсказания класса. 

После определения  функции `two_layer_fc` мы проверим её реализацию, исполнив вычислительный граф, подав нули на входы сети и проверив формы выходных двнных.

Важно, чтобы вы прочитали и поняли эту реализацию.

In [8]:
def two_layer_fc(x, params):
    """
    Полносвязная 2-х слойная нейронная сеть; 
    архитектура: полносвязный слой -> ReLU ->  полносвязный слой .
    
    Обратите внимание, что нам сейчас нужно только определить 
    прямое распространение;TensorFlow позаботится сам о
    вычислении градиентов.
    
    Вход сети - мини-блок данных размерности (формы):
    (N, d1, ..., dM), где d1 * ... * dM = D. 
    Скрытый слой содержит H нейронов.
    Выходной слой вычисляет оценки рейтигов для C классов.

    Входы:
     - x: тензор формы (N, d1, ..., dM), представляющий мини-блок
       входных данных.
     - params: список [w1, w2] тензоров, представляющих веса сети,
       где w1 имеет форму (D, H), а w2 имеет форму (H, C).
    
     Возвращает:
     - scores: тензор формы (N, C), представляющий оценки рейтингов 
       принадлежности классам входных данных x.  
    """
    
    w1, w2 = params                   # распаковка параметров
    x = flatten(x)                    # реформатируем x к форме (N, D)
    h = tf.nn.relu(tf.matmul(x, w1))  # Скрытый слой : форма h - (N, H)
    scores = tf.matmul(h, w2)         # Вычисление рейтингов, форма scores - (N, C)
    return scores

In [9]:
def two_layer_fc_test():
    """ Функция проверки двухслойной сети"""
    hidden_layer_size = 42

    # Разместим  код вычислительного графа в контексте менежджера устройств tf.device,
    # что позволит указывать TensorFlow, где должны размещаться тензоры в CPU или GPU
    with tf.device(device):        
        x = tf.zeros((64, 32, 32, 3))
        w1 = tf.zeros((32 * 32 * 3, hidden_layer_size))
        w2 = tf.zeros((hidden_layer_size, 10))

        # Вызов функции two_layer_fc для реализации прямого распространения 
        scores = two_layer_fc(x, [w1, w2])

    print('Форма выходных данных 2х слойной сети:',scores.shape) #Вывод размерности score

two_layer_fc_test()

Форма выходных данных 2х слойной сети: (64, 10)


### Базовый TensorFlow: Трехслойная  ConvNet

Реализуйте функцию `three_layer_convnet`, которая будет выполнять прямое распространение для трехслойной сверточной сети. Сеть  должна иметь следующую архитектуру:

1. Сверточный слой (со смещением) с числом фильтров `channel_1`, каждый размером ` KW1 x KH1` и дополнением двумя нулями, P=2
2. Нелинейность ReLU
3. Сверточный слой (со смещением) с числом фильтров `channel_2`, каждый размером ` KW2 x KH2` и  дополнением одним нулем, P=1
4. Нелинейность ReLU
5. Полносвязанный слой со смещением, вычисляющий оценки рейтингов для `C` классов .


**СОВЕТ**: Для реализации свертки используйте вызов: `tf.nn.conv2d(x,filter,strides,padding,use_cudnn_on_gpu = None,…)`, где параметр filter - список вида `[Н,W,Channel_in,Channel_out]`; strides - 4-х элементный список значений сдвига окна свертки в каждом направлении, часто `[1,1,1,1]`; `padding ='VALID'`.

Будьте внимательны с добавлением нулей! Нули добавляем только по высоте и ширине изображения. Например, дополнение двумя нулями по высоте и ширине можно реализовать так:   `tf.pad(x, [[0, 0], [2, 2], [2, 2], [0, 0]], 'CONSTANT')`.


In [10]:
def three_layer_convnet(x, params):
   
    """
    Трехслойная сверточная сеть с описанной выше архитектурой.
    
    Входы:
    - x: тензор формы (N, H, W, 3), представляющий мини-блок изображений
    - params: список тензоров, представляющих веса и смещения 
      сети; должен содержать следующее:
      - conv_w1: тензор формы (KH1, KW1, 3, channel_1) -
        веса первого сверточного слоя.
      - conv_b1: тензор формы (channel_1,) - смещения
        первого сверточного слоя.
      - conv_w2: тензор формы (KH2, KW2, channel_1, channel_2) -
        веса второго сверточного слоя
      - conv_b2: тензор  формы (channel_2,) - смещения
        второго сверточного слоя.
      - fc_w: тензор представляющий весовые коэффициенты полносвязанного слоя.
        Могли бы определить сами, какая должна быть форма?
      - fc_b: тензор смещений полносвязанного слоя.
        Могли бы определить сами, какая должна быть форма?    
    """

    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    
    ############################################################################
    # ЗАДАНИЕ: Реализуйте прямое распространение для 3-х слойной ConvNet.      #
    ############################################################################
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
   
    # дополняем x двумя нулями по высоте и ширине
    x_padded = tf.pad(x, [[0, 0], [2, 2], [2, 2], [0, 0]], 'CONSTANT')
    
    # реализуем первый свёрточный слой +relu
    conv1 = tf.nn.conv2d(x_padded, conv_w1, [1, 1, 1, 1], padding='VALID') + conv_b1
    relu1 = tf.nn.relu(conv1)
    
    # дополняем выход relu нулями по высоте и ширине
    conv1_padded = tf.pad(relu1, [[0, 0], [1, 1], [1, 1], [0, 0]], 'CONSTANT')
    
    # реализуем второй свёрточный слой +relu
    conv2 = tf.nn.conv2d(conv1_padded, conv_w2, [1, 1, 1, 1], padding='VALID') + conv_b2
    relu2 = tf.nn.relu(conv2)
    
    # 'уплощаем' выход relu2 (преобразуем в вектор)
    conv2_flattened = flatten(relu2)
    
    # реализуем полносвязанный слой
    fc1 = tf.matmul(conv2_flattened, fc_w) + fc_b
    scores = fc1

    # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ############################################################################
    #                              КОНЕЦ ВАШЕГО КОДА                           #
    ############################################################################
    
    return scores

После определения  трехслойной ConvNet выше, запустите следующую ячейку, чтобы проверить вашу реализацию. Подобно двухслойной сети выполним проверку на миниблоке данных из нулей только для того, чтобы убедиться, что функция не дает сбоев, и создает выходы правильной формы (размерности).

В результате выполнения этой функции `scores_np` должен будет иметь форму` (64, 10) `.

In [11]:
def three_layer_convnet_test():
    
    with tf.device(device):
        x = tf.zeros((64, 32, 32, 3))      #N=64  HхW=32x32,  C=3
        conv_w1 = tf.zeros((5, 5, 3, 6))   #KH1,KW1,3,ch1
        conv_b1 = tf.zeros((6,))
        conv_w2 = tf.zeros((3, 3, 6, 9))   #KH2,KW2,ch1,ch2
        conv_b2 = tf.zeros((9,))
        fc_w = tf.zeros((32 * 32 * 9, 10)) #32x32xch2
        fc_b = tf.zeros((10,))
        params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
        scores = three_layer_convnet(x, params)

    # Входы  сверточных слоев представляют собой 4-мерные массивы с формой
    # [размер_блока, высота, ширина, каналы]
    print('Форма выходных данных сверточной сети:', scores.shape)

three_layer_convnet_test()

Форма выходных данных сверточной сети: (64, 10)


### Базовый TensorFlow: шаг обучения

Теперь мы определим функцию `training_step`, которая выполняет один шаг обучения. Для этого необходимо выполнить три основных действия:

1. Вычислить функцию потерь;
2. Вычислить градиент функции потерь по отношению ко всем весам сети;
3. Выполнить шаг обновления весов, используя стохастический градиентный спуск.

Чтобы сделать все это, нам нужно использовать несколько новых функций и объектов TensorFlow:
1. Для вычисления кросс-энтропийной функции потерь будем использовать метод `tf.nn.sparse_softmax_cross_entropy_with_logits(labels,logits)`, который  вычисляет softmax кросс-энтропию между вектором меток labels (размер (N, )) и  массивом logits (размер (N, C)) не масштабированных log вероятностей;
2. Для усреднения потерь по мини-блоку данных  будем использовать метод `tf.math.reduce_mean(input_tensor, axis=None,  keepdims=False)`, который вычисляет среднее по измерениям входного тензора, axis – указывает по каким осям усреднять (None – по всем), keepdims=True – размеры сокращаются до 1);
3.  Для вычисления градиентов функции потерь по отношению к весам будем использовать объект `tf.GradientTape()` , в области действия которого записывают те операции, которые образуют вычислительный граф нейросети;  по крайней мере, один из аргументов этих операций  должен быть наблюдаемым ("watched"); обучаемые переменные, созданные с помощью  `tf.Variable`  (где trainable=True по умолчанию) являются автоматически наблюдаемыми; в контекст `tf.GradientTape()` включают вызов метода `gradient` для вычисления производных.
4. Будем изменять значения весов, хранящихся в виде тензоров TensorFlow, используя `tf.assign_sub(ref, value)` – обновляет тензор ref путем вычитания value


In [12]:
def training_step(model_fn, x, y, params, learning_rate):
    """
     Функция, реализующая шаг SGD обучения модели model_fn
     Входы:
     - model_fn: функция модели нейросети, которая реализует прямое распространение,
       используя TensorFlow; она должна иметь следующую сигнатуру:
       scores = model_fn (x, params), где x - тензор, представляющий
       мини-блок данных с изображениями, params - список тензоров, хранящих
       веса модели, scores - тензор  формы (N, C), содержащий
       рейтинги  всех классов из x.
     - x: входной тензор модели model_fn
     - y: тензор, содержащий корректные метки классов
     - params: cписок тензоров, представляющих  параметры модели
     - learning_rate: скорость обучения SGD
      Возвращает: total_loss – общие потери на блоке данных x
    """
    with tf.GradientTape() as tape:
        scores = model_fn(x, params) # Прямое распространение
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        total_loss = tf.reduce_mean(loss) # Вычисление средних потерь
        grad_params = tape.gradient(total_loss, params) # Вычисление градиентов по параметрам

        # Выполнение шага SGD по всем параметрам модели
        # Ручное обновление весов с помощью assign_sub()
        # zip формирует список кортежей из пар параметров
        for w, grad_w in zip(params, grad_params):
            w.assign_sub(learning_rate * grad_w)
                        
        return total_loss
        

In [13]:
def train_part2(model_fn, init_fn, learning_rate):
    """
    Обучение модели на множестве CIFAR-10.
        
    Входы:
     - model_fn: функция модели нейросети, которая реализует прямое распространение,
       используя TensorFlow; она должна иметь следующую сигнатуру:
       scores = model_fn (x, params), где x - тензор, представляющий
       мини-блок данных с изображениями, params - список тензоров, хранящих
       веса модели, scores - тензор  формы (N, C), содержащий
       рейтинги  всех классов из x.
     - init_fn: функция, которая инициализирует параметры модели.
       Она должна иметь сигнатуру params = init_fn(), где params - это список
       тензоров, хранящих (случайно инициализированные) веса модели.
     - learning_rate: скорость обучения.
      
    """
    
    params = init_fn()  # Инициализация параметров модели            
    # цикл по множеству обучающих данных    
    for t, (x_np, y_np) in enumerate(train_dset):
        # Выполнение шага обучения на блоке обучающих данных
        loss = training_step(model_fn, x_np, y_np, params, learning_rate)
        
        # Периодически выводим потери и проверяем точность на валидационном множестве
        if t % print_every == 0:
            print('Итерации %d, потери = %.4f' % (t, loss))
            #check_accuracy(val_dset, x_np, model_fn, params)
            check_accuracy(val_dset, model_fn, params)

In [14]:
def check_accuracy(dset, model_fn, params):
    """
    Проверяет точность классификации.
    
     Входы:
     - dset: данные типа Dataset, используемые для проверки точности
     - model_fn: модель, которую мы будем вызывать для предказания по x
     - params: параметры модели  model_fn
      
     Возвращает: ничего не возвращает, но выводит точность модели
    """
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        scores_np = model_fn(x_batch, params).numpy()
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Классифицировано %d / %d корректно (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### Базовый TensorFlow: инициализация весов нейросети
Мы будем использовать функцию для инициализации матриц весов моделей, базирующуюся на методе
нормировки, предложенном в

[1] He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification
*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [15]:
def kaiming_normal(shape):   
    if len(shape) == 2:                  # для полносвязной сети
        fan_in, fan_out = shape[0], shape[1]
    elif len(shape) == 4:                # для сверточной сети
        fan_in, fan_out = np.prod(shape[:3]), shape[3]
    # веса масштабируются обратно-пропорционально квадратному корню от "число входов/2"   
    return tf.keras.backend.random_normal(shape) * np.sqrt(2.0 / fan_in)

### Базовый  TensorFlow: обучение 2-х слойной нейросети
Наконец, мы готовы использовать все части, определенные выше, для обучения двухслойной полносвязной сети на множестве данных CIFAR-10.

Нам просто нужно определить функцию для инициализации весов модели и вызвать `train_part2`.

Определение весов сети представляет собой еще один важный компонент TensorFlow API: `tf.Variable`. TensorFlow Variable - это тензор-переменная, значение которой хранится в графе и сохраняется на разных  циклах исполнения вычислительного графа; однако в отличие от констант, определенных с помощью `tf.zeros` или` tf.random_normal`, значения переменной могут быть изменены при выполнении графа; эти изменения будут сохраняться в графе. Обучаемые параметры сети обычно хранятся в переменных.

Вам не нужно настраивать гиперпараметры, но вы должны достичь точности выше 40% после одной эпохи обучения.

In [16]:
def two_layer_fc_init():
    """
    Инициализирует веса двухслойной сети для использования с
    two_layer_network, определенной выше.
    
     Входы: отсутствуют
    
     Возвращает: список:
     - w1: TensorFlow tf.Variable, представляющая веса первого слоя
     - w2: TensorFlow tf.Variable, представляющая веса второго слоя
    """
    hidden_layer_size = 4000
    w1 = tf.Variable(kaiming_normal((3 * 32 * 32, 4000)))
    w2 = tf.Variable(kaiming_normal((4000, 10)))
    return [w1, w2]

# Обучение двухслойной сети
learning_rate = 1e-2
train_part2(two_layer_fc, two_layer_fc_init, learning_rate)

Итерации 0, потери = 3.2034
Классифицировано 100 / 1000 корректно (10.00%)
Итерации 100, потери = 2.0725
Классифицировано 371 / 1000 корректно (37.10%)
Итерации 200, потери = 1.5269
Классифицировано 393 / 1000 корректно (39.30%)
Итерации 300, потери = 1.8327
Классифицировано 370 / 1000 корректно (37.00%)
Итерации 400, потери = 1.7878
Классифицировано 423 / 1000 корректно (42.30%)
Итерации 500, потери = 1.7353
Классифицировано 429 / 1000 корректно (42.90%)
Итерации 600, потери = 1.8410
Классифицировано 429 / 1000 корректно (42.90%)
Итерации 700, потери = 1.9588
Классифицировано 442 / 1000 корректно (44.20%)


### Базовый  TensorFlow: Обучение 3-х слойной  ConvNet

Теперь мы будем использовать TensorFlow для обучения трехслойной ConvNet на CIFAR-10.

Вам нужно реализовать функцию `three_layer_convnet_init`. Напомним архитектуру сети:

1. Сверточный слой (со смещением) с 32 фильтрами 5 × 5 с дополнением нулями Р=2
2. ReLU
3. Сверточный слой (со смещением) с 16 фильтрами 3x3 с дополнением нулями Р=1
4. ReLU
5. Полносвязный слой (со смещением) для вычисления оценок scores 10 классов

Вам не нужно делать какие-либо настройки гиперпараметров, но вы должны получить точность выше 43% после одной эпохи обучения.

In [17]:
def three_layer_convnet_init():
    """
    Инициализирует веса трехслойной ConvNet, для использования с
    three_layer_convnet, определенной выше.
    Вы можете использовать `kaiming_normal`!
    
     Входы: Отсутствуют
    
     Возвращает список, содержащий:
     - conv_w1: переменная tf.Variable, содержащая веса для первого слоя conv
     - conv_b1: переменная tf.Variable, содержащая смещения для первого слоя conv
     - conv_w2: переменная tf.Variable, содержащая веса для второго слоя conv
     - conv_b2: переменная tf.Variable, содержащая смещения для второго слоя conv
     - fc_w: переменная tf.Variable, содержащая веса  для полносвязанного слоя
     - fc_b: переменная tf.Variable, содержащая смещения для полносвязанного слоя
    """
    
    params = None
    
    ############################################################################
    # ЗАДАНИЕ: Инициализаровать параметры 3-х слойной сети                     #
    ############################################################################
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    
    conv_w1 = tf.Variable(kaiming_normal((5, 5, 3, 32)))
    conv_b1 = tf.Variable(tf.zeros([32]))
    conv_w2 = tf.Variable(kaiming_normal((3, 3, 32, 16)))
    conv_b2 = tf.Variable(tf.zeros([16]))
    fc_w = tf.Variable(kaiming_normal((32 * 32 * 16, 10)))
    fc_b = tf.Variable(tf.zeros([10]))
    params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]

    # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ############################################################################
    #                              КОНЕЦ ВАШЕГО КОДА                           #
    ############################################################################
    
    return params

# Обучение трехслойной сверточной модели
learning_rate = 3e-3
train_part2(three_layer_convnet, three_layer_convnet_init, learning_rate)

Итерации 0, потери = 2.7652
Классифицировано 101 / 1000 корректно (10.10%)
Итерации 100, потери = 1.9594
Классифицировано 354 / 1000 корректно (35.40%)
Итерации 200, потери = 1.5008
Классифицировано 381 / 1000 корректно (38.10%)
Итерации 300, потери = 1.6558
Классифицировано 395 / 1000 корректно (39.50%)
Итерации 400, потери = 1.7575
Классифицировано 431 / 1000 корректно (43.10%)
Итерации 500, потери = 1.7014
Классифицировано 434 / 1000 корректно (43.40%)
Итерации 600, потери = 1.6804
Классифицировано 453 / 1000 корректно (45.30%)
Итерации 700, потери = 1.6238
Классифицировано 448 / 1000 корректно (44.80%)


# Часть III: Использование API объекта tf.keras.Model
Реализация нейронной сети с использованием базового API TensorFlow - это хороший способ понять, как работает TensorFlow, но  не удобно - нам пришлось вручную отслеживать все тензоры, представляющие обучаемые параметры. Это не сложно для небольшой сети, но усложняется при большой модели нейросети.

К счастью, TensorFlow 2.0 имеет API  высокого уровня, такой как `tf.keras`, который упрощает создание моделей из модульных объектно-ориентированных слоев. Кроме того, TensorFlow 2.0 использует  режим "eager" выполнения, которое выполняет операции немедленно. Это облегчает написание и отладку моделей и сокращает стандартный код. 

В этой части блокнота мы определим модели нейронных сетей, используя API интерфейс высокого уровня `tf.keras.Model`. Чтобы реализовать свою собственную модель, вам необходимо сделать следующее:

1. Определите новый класс, который является подклассом `tf.keras.model`. Присвойте вашему классу соответствующее имя, которое указывает его назначение, например `TwoLayerFC` или `ThreeLayerConvNet`.
2. В конструкторе `__init __ ()` нового класса определите все слои, которые вам нужны, в виде атрибутов класса. Модуль `tf.keras.layers` предоставляет множество обобщенных нейросетевых слоёв, таких как `tf.keras.layers.Dense` для полносвязанных слоев или  `tf.keras.layers.Conv2D` для сверточных слоев. Внутри эти слои будут автоматически создавать  `Variable` тензоры  для любых обучаемых параметров. **Предупреждение**: Не забудьте вызвать `super(YourModelName, self).__init__()`  в качестве первой строки конструктора нового класса!
3. Реализуйте метод `call ()` для вашего класса; он осуществляет прямое распространение для вашей модели и определяет *связи*  слоев вашей сети. Слои, определенные в `__init __ ()`, применяются в  `__call __ ()`,  виде функций, которые преобразуют входные тензоры в выходные тензоры. Не определяйте новые слои в `call ()`; любые слои, которые вы хотите использовать при прямом распространении, должны быть определены в `__init __ ()`.

После того, как вы определили свой подкласс `tf.keras.Model`, вы можете создать его экземпляр и использовать его подобно модели из части II.

### Создание  модели 2-х слойной сети в виде подкласса tf.keras.Model

Ниже приведен конкретный пример использования API `tf.keras.Model` для определения двухслойной сети. 

Мы используем объект `Initializer` для задания начальных значений обучаемых параметров слоев; в частности, `tf.initializers.VarianceScaling` соответствует методу инициализации, использованному в функции `kaiming_normal` в части II. 

Объект `tf.keras.layers.Dense`  используется для представления двух полносвязанных слоев модели. В дополнение к умножению входа на весовую матрицу и добавлению вектора смещения, этот слой также может обеспечить применение нелинейности. Для первого слоя используется функция активации ReLU, для этого конструктору передается параметр `activation='relu'`; второй слой использует `softmax` функцию активации. В заключение мы определяем слой `tf.keras.layers.Flatten` для "уплощения" выхода предыдущего слоя.

In [18]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()  
        # определяем функцию-инициализатор весов (нормальное распределение с дисперсией = sqrt(scale /fan_in))
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        # описываем набор слоев сети 
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu', 
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        # описываем порядок взаимодействия слоев при прямом распространении
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ Небольшой тест для модели TwoLayerFC"""
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    # создаем экземляр класса - модель 2-х слойной сети
    model = TwoLayerFC(hidden_size, num_classes)
    # выполняем прямое распространение 
    with tf.device(device):
        scores = model(x)
        print(scores.shape) # просто проверяем форму выходных данных
        
test_TwoLayerFC()

(64, 10)


### Создание  модели 3-х слойной Conv сети в виде подкласса tf.keras.Model 
Реализуйте трехслойную сверточную сеть ConvNet с использованием API `tf.keras.Model`. Модель должна иметь ту же архитектуру, что и ранее в части II:

1. Сверточный слой с 5 х 5 фильтрами и  с дополнением нулями Р=2;
2. Нелинейность ReLU;
3. Сверточный слой с 3 x 3 фильтрами и  с дополнением нулями Р=1;
4. Нелинейность ReLU;
5. Полносвязный слой, формирующий рейтинги  классов scores;
6. Softmax нелинейность.

Вы должны инициализировать веса сети, используя тот же метод инициализации, который использовался в двухслойной сети выше.

**Совет**: обратитесь к документации для `tf.keras.layers.Conv2D` и `tf.keras.layers.Dense`


In [19]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        
        ########################################################################
        # ЗАДАНИЕ:                                                             #
        # экземпляры объектов слоя,используемые при прямом распространении     #
        # Реализуйте метод __init__ для трехслойной ConvNet. Вы должны создать #    
        ########################################################################
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        
        # 1. Определяем слои модели внутри конструктора init
        
        # выбираем инициализатор весов слоев
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        
        # определяем слои модели, содержащие обучаемые параметры
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), strides=1, padding='valid',
                                           kernel_initializer=initializer, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), strides=1, padding='valid',
                                           kernel_initializer=initializer, activation='relu')
        self.fc = tf.keras.layers.Dense(num_classes, kernel_initializer=initializer, 
                                        activation='softmax')

        # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        ########################################################################
        #                             КОНЕЦ ВАШЕГО КОДА                        #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        
        ########################################################################
        # ЗАДАНИЕ: выполнить прямое распространение для 3-х слойной ConvNet.   #
        # Используйте объекты слоя, определенные в методе __init__.            #
        ########################################################################
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        
        # 2. Определяем связи слоев внутри метода call подкласса
                
        # дополняем x нулями вручную и проходим слой conv1
        padding = tf.constant([[0, 0], [2, 2], [2, 2], [0, 0]])
        x = tf.pad(x, padding, 'CONSTANT')
        x = self.conv1(x)
        
        # дополняем x нулями вручную и проходим слой conv2
        padding = tf.constant([[0, 0], [1, 1], [1, 1], [0, 0]])
        x = tf.pad(x, padding, 'CONSTANT')
        x = self.conv2(x)
        
        # прямое прохождение fc слоя
        x = tf.keras.layers.Flatten()(x)
        scores = self.fc(x)

        # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        ########################################################################
        #                             КОНЕЦ ВАШЕГО КОДА                        #
        ########################################################################
        
        return scores

После завершения реализации `ThreeLayerConvNet`  выполните код в ячейке ниже, чтобы убедиться, что ваша реализация  возвращает выходы ожидаемой формы.

In [20]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


### Организация собственного цикла обучения для моделей в виде подкласса tf.keras.Model (использование tf. GradientTape) 

Keras имеет простой встроенный цикл обучения (реализуется посредством `model.fit`), но иногда вам требуется большая гибкость  настроек процесса обучения. Ниже приведен пример цикла обучения, реализованного с использованием безотлагательного (eager) исполнения.

Для  реализации такого цикла используется объект `tf.GradientTape`, который называется лентой (tape). В основе TensorFlow заложено автоматическое дифференцирование  для реализации обратного распространения. В ходе безотлагательного исполнения tf.GradientTape используется для трассировки операций, которые позже будут применены для вычисления градиентов. 

TensorFlow 2.0 поставляется с простыми в использовании встроенными метриками в модуле `tf.keras.metrics`. Каждая метрика является объектом, и мы можем использовать `update_state ()` для обновления состояний метрик и `reset_state ()` для очистки всех состояний. Мы можем получить текущее значение метрики, вызвав `result ()` для объекта-метрики.

In [21]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Функция, реализующая простой цикл обучения для использования с моделями, 
    определенными с помощью tf.keras. Она обучает модель на одной эпохе на 
    обучающем мн-ве CIFAR-10 и периодически проверяет точность на валидационном
    мн-ве CIFAR-10.
    
     Входы:
     - model_init_fn: функция создает модель, которую мы хотим обучить: 
                      model = model_init_fn ()
     - optimizer_init_fn: функция создает объект Optimizer, который мы будем 
                          использовать для оптимизации модели: optimizer = optimizer_init_fn ()
     - num_epochs: количество эпох обучения
    
     Возвращает: ничего не возвращает, но отображает ход обучения
      
    """
    
    with tf.device(device):
        
        # Создаем экземпляр объекта для вычисления кросс-энтропийных потерь
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        # Cоздаем экземпляры объектов модели и оптимизатора
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        # Создаем экземпляры метрик для этапов обучения и валидации
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0 # номер шага итерации    
        for epoch in range(num_epochs):
            
            # Сброс состояний метрик этапа обучения
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            # Итерации по обучающему множеству
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape: 
                    
                    # Используем model для вычислений прямого пути и потерь
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
                    
                    # Вычисляем градиенты и обновляем обучаемые переменные
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Обновляем метрики
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    # Мониторинг процесса обучения через заданное число итераций
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        
                        # Предсказание и оценка потерь на множестве валидации
                        for test_x, test_y in val_dset:
                            # В ходе валидации  устанавливаем training в False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)
                            # Обновляем состояния метрик валидации
                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Итерация {}, Эпоха {}, Потери: {}, Точность: {}, Val Потери: {}, Val Точность: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

### Обучение двухслойной модели, реализованной в виде подкласса  tf.keras.Model
Теперь мы можем использовать описанные выше инструменты для обучения двухслойной сети на множестве CIFAR-10. При вызове модели необходимо определить функции `model_init_fn` и` optimizer_init_fn`, которые задают используемую модель и оптимизатор, соответственно. Обучение модели будем выполнять с использованием стохастического градиентного спуска, поэтому в качестве оптимизатора используем функцию `tf.keras.optimizers.SGD`. 

Здесь не требуется настраивать какие-либо гиперпараметры, но вы должны достичь точности валидации выше 40% после одной эпохи обучения. 

In [22]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Итерация 0, Эпоха 1, Потери: 3.2795963287353516, Точность: 7.8125, Val Потери: 2.9306931495666504, Val Точность: 10.699999809265137
Итерация 100, Эпоха 1, Потери: 2.2585976123809814, Точность: 27.923885345458984, Val Потери: 1.8452696800231934, Val Точность: 38.10000228881836
Итерация 200, Эпоха 1, Потери: 2.090008020401001, Точность: 31.941852569580078, Val Потери: 1.8430598974227905, Val Точность: 40.099998474121094
Итерация 300, Эпоха 1, Потери: 2.0071892738342285, Точность: 33.79360580444336, Val Потери: 1.8492001295089722, Val Точность: 36.70000076293945
Итерация 400, Эпоха 1, Потери: 1.936073660850525, Точность: 35.750465393066406, Val Потери: 1.6857361793518066, Val Точность: 42.89999771118164
Итерация 500, Эпоха 1, Потери: 1.8909555673599243, Точность: 36.76708984375, Val Потери: 1.6346993446350098, Val Точность: 43.5
Итерация 600, Эпоха 1, Потери: 1.8622238636016846, Точность: 37.69239044189453, Val Потери: 1.6826175451278687, Val Точность: 42.69999694824219
Итерация 700, Эпох

### Обучение трехслойной модели ConvNet, реализованной в виде подкласса  tf.keras.Model
Здесь необходимо использовать инструменты, которые мы определили выше для обучения трехслойной сверточной сети ConvNet на множестве  CIFAR-10. Модель ConvNet должна содержать 32 фильтра в первом сверточном слое и 16 фильтров во втором слое.

Для обученияи модели следует использовать алгоритм градиентного спуска с моментом Нестерова с  коэффициентом затухания  0,9.

Вам не требуется выполнять какую-либо настройку гиперпараметров, но вы должны достичь точности вадидации более 50% после одной эпохи обучения.

In [23]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    
    ############################################################################
    # ЗАДАНИЕ: определите  model в виде 3-х слойной conv сети                  #
    ############################################################################
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
   
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ############################################################################
    #                           КОНЕЦ ВАШЕГО КОДА                              #
    ############################################################################
    
    return model

def optimizer_init_fn():
    optimizer = None
    
    ############################################################################
    # ЗАДАНИЕ: Вызовите функцию, определящую optimizer                         #
    ############################################################################
    # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
   
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                       momentum=0.9, nesterov=True)

    # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
    ############################################################################
    #                           КОНЕЦ ВАШЕГО КОДА                              #
    ############################################################################
    
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Итерация 0, Эпоха 1, Потери: 2.5686373710632324, Точность: 12.5, Val Потери: 3.1889030933380127, Val Точность: 8.0
Итерация 100, Эпоха 1, Потери: 1.908718466758728, Точность: 33.02908706665039, Val Потери: 1.6891686916351318, Val Точность: 41.400001525878906
Итерация 200, Эпоха 1, Потери: 1.74983811378479, Точность: 38.53389358520508, Val Потери: 1.5006937980651855, Val Точность: 49.099998474121094
Итерация 300, Эпоха 1, Потери: 1.6562060117721558, Точность: 41.66839599609375, Val Потери: 1.4256306886672974, Val Точность: 51.099998474121094
Итерация 400, Эпоха 1, Потери: 1.5833784341812134, Точность: 44.08510208129883, Val Потери: 1.3670945167541504, Val Точность: 51.400001525878906
Итерация 500, Эпоха 1, Потери: 1.5328413248062134, Точность: 45.8863525390625, Val Потери: 1.3284120559692383, Val Точность: 53.79999923706055
Итерация 600, Эпоха 1, Потери: 1.498449683189392, Точность: 47.04398727416992, Val Потери: 1.3077772855758667, Val Точность: 54.29999923706055
Итерация 700, Эпоха 1,

# Часть IV:  Последовательный и функциональные API Keras
В третьей части мы представили API `tf.keras.Model`, который позволяет Вам определять модели с любым количеством доступных для обучения слоев и с произвольными связями между слоями.

Однако для многих случаев такая гибкость модели не требуется - многие модели могут быть выражены в виде последовательного стека слоев, при этом выходные данные каждого слоя передаются на следующий уровень в качестве входных данных. Если ваша модель соответствует этому шаблону, то существует еще более простой способ определить вашу модель: использовать `tf.keras.Sequential`. Вам не нужно писать какие-либо пользовательские классы; Вы просто вызываете конструктор `tf.keras.Sequential` со списком, содержащим последовательность объектов слоя.

Одна сложность с `tf.keras.Sequential` заключается в том, что вы должны определить форму входных данных для модели, передав значение` input_shape` первого слоя в вашей модели.

### Keras Sequential API: двухслойная сеть
В этом подразделе мы перепишем полносвязанную двухслойную сеть, используя `tf.keras.Sequential`, и обучим ее, используя обучающий цикл, определенный выше.

Здесь не требуется выполнять какую-либо настройку гиперпараметров, но вы должны достичь точности валидации более 40% после одной эпохи обучения.

In [24]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    # список слоев модели
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    # coздаем модель в виде экземпляра класса tf.keras.Sequential
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Итерация 0, Эпоха 1, Потери: 2.88728404045105, Точность: 9.375, Val Потери: 2.922335147857666, Val Точность: 12.600000381469727
Итерация 100, Эпоха 1, Потери: 2.2514092922210693, Точность: 28.29517364501953, Val Потери: 1.9011563062667847, Val Точность: 39.0
Итерация 200, Эпоха 1, Потери: 2.0886340141296387, Точность: 32.182838439941406, Val Потери: 1.8302842378616333, Val Точность: 40.400001525878906
Итерация 300, Эпоха 1, Потери: 2.0088131427764893, Точность: 34.07910919189453, Val Потери: 1.8670812845230103, Val Точность: 39.099998474121094
Итерация 400, Эпоха 1, Потери: 1.9351927042007446, Точность: 35.86346435546875, Val Потери: 1.733605146408081, Val Точность: 41.20000076293945
Итерация 500, Эпоха 1, Потери: 1.8898835182189941, Точность: 37.0228271484375, Val Потери: 1.6667338609695435, Val Точность: 41.29999923706055
Итерация 600, Эпоха 1, Потери: 1.8600142002105713, Точность: 37.94717025756836, Val Потери: 1.7060847282409668, Val Точность: 41.900001525878906
Итерация 700, Эпоха

### Keras: Обучение  Sequential  модели с помощью встроенного метода model.fit

В предыдущих примерах мы использовали собственный настраиваемый цикл обучения для обучения моделей ( `train_part34`). Написание собственного цикла обучения требуется только в том случае, если вам нужна большая гибкость и контроль во время обучения Вашей модели. Альтернативно, Вы также можете использовать встроенные API, такие как `tf.keras.Model.fit ()` и `tf.keras.Model.evaluate`, чтобы обучать и оценивать модель. Также не забывайте  настраивать свою модель для обучения, вызвав `tf.keras.Model.compile`.

Здесь не требуется выполнять какую-либо настройку гиперпараметров, но вы должны достичь точности вадидации более 42% после одной эпохи обучения.

In [25]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

Train on 49000 samples, validate on 1000 samples
10000/10000 [==============================] - 4s 353us/sample - loss: 1.5909 - sparse_categorical_accuracy: 0.4564


[1.590888639831543, 0.4564]

## Функциональный API:
### Демонстрация двухслойной сети

В предыдущем разделе видели, как мы можем использовать `tf.keras.Sequential` для быстрого построения простых моделей. Но это достигается за счет потери гибкости.

Часто нам приходится писать сложные модели, которые имеют непоследовательные потоки данных: у слоя может быть **несколько входов и / или выходов**, таких как объединение выходных данных двух предыдущих слоев для подачи на вход третьего слоя! (примеры: шунтирование слоев и полносвязанные  блоки.)

В таких случаях мы можем использовать функциональный API Keras для написания моделей со сложными топологиями, такими как:

  1. Модели с несколькими входами
  2. Модели с несколькими выходами
  3. Модели с общими слоями (один и тот же слой вызывается несколько раз)
  4. Модели с непоследовательными потоками данных (например, шунтирующие соединения)

Написание модели с помощью Functional API требует от нас создания экземпляра `tf.keras.Model` и явной записи входных и выходных тензоров в виде параметров слоев для этой модели.

In [26]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    # описание слоев в виде функций с аргументами
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Создание экземпляра модели с заданными входами и выходами
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ Небольшой тест для модели TwoLayerFC, определенной выше  """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


### Функциональный API Keras: обучение двухслойной сети
Теперь вы можете обучить эту двухслойную сеть, построенную с использованием функционального API.

Вам не нужно выполнять выбор гиперпараметров, но вы должны достичь валидационной точности выше 40% после обучения в течение одной эпохи.

In [27]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Итерация 0, Эпоха 1, Потери: 2.9675796031951904, Точность: 17.1875, Val Потери: 2.992048501968384, Val Точность: 12.800000190734863
Итерация 100, Эпоха 1, Потери: 2.2617239952087402, Точность: 28.418933868408203, Val Потери: 1.8459010124206543, Val Точность: 38.900001525878906
Итерация 200, Эпоха 1, Потери: 2.092485189437866, Точность: 32.26834487915039, Val Потери: 1.8734703063964844, Val Точность: 38.400001525878906
Итерация 300, Эпоха 1, Потери: 2.010263681411743, Точность: 34.131019592285156, Val Потери: 1.8420852422714233, Val Точность: 38.400001525878906
Итерация 400, Эпоха 1, Потери: 1.9396835565567017, Точность: 35.97646713256836, Val Потери: 1.7187179327011108, Val Точность: 42.599998474121094
Итерация 500, Эпоха 1, Потери: 1.8951425552368164, Точность: 36.979164123535156, Val Потери: 1.6734684705734253, Val Точность: 43.20000076293945
Итерация 600, Эпоха 1, Потери: 1.8634939193725586, Точность: 37.902976989746094, Val Потери: 1.6827170848846436, Val Точность: 41.8000030517578

# Часть IV: CIFAR-10 - открытая задача

В этом разделе Вы можете поэкспериментировать с любой архитектурой ConvNet, которую Вы хотели бы использовать для клаcсификации изображений множества CIFAR-10.

Вы должны поэкспериментировать с архитектурами, гиперпараметрами, функциями потерь, регуляризацией или чем-либо еще, что Вы посчитаете важным для обучения модели, которая достигает **не менее 70%** точности на **валидационном** множестве в течение 10 эпох. Вы можете использовать встроенную функцию обучения, функцию `train_part34`, определенную выше, или Вы можете реализовать свой собственный цикл обучения.

Опишите Ваши эксперименты в конце этого блокнота.

### С чем Вам следует экспериментировать:
- **Размер фильтра**: Выше мы использовали 5x5 и 3x3; это оптимально?
- **Количество фильтров**: Выше мы использовали 16 и 32 фильтра. Будет ли большее или меньшее число фильтров лучше?
- **Пулинг**: Мы не использовали никакого пулинга выше. Будет ли пулинг улучшать модель?
- **Нормализация**: улучшится ли ваша модель при использовании блочной нормализации, нормализации на слое, нормализации группы или какой-либо иной стратегии нормализации?
- **Архитектура**: В приведенной выше ConvNet имеется только три слоя обучаемых параметров. Будет ли более глубокая модель работать лучше?
- **Глобальный усредняющий пулинг**: вместо "уплощения" данных после последнего сверточного слоя будет ли глобальный усредняющий пулинг лучше? Эта стратегия используется, например, в  сети Google Inception  и в Остаточных (Residual) сетях.
- **Регуляризация**: Будет ли какая-то регуляризация повышать эффективность сети? Может быть, затухание весов или dropout?

### Замечание: Блочная нормализация/ Dropout
Если вы используете Batch Normalization и Dropout, не забудьте передать `is_training = True`, если вы используете функцию` train_part34 () `. Слои BatchNorm и Dropout ведут себя по-разному во время обучения и предсказания. `training` - это специальный ключевой аргумент, зарезервированный для этой цели в любой `tf.keras.Model` функции` call () `. 

### Подсказки для обучения
Для каждой сетевой архитектуры, с которой Вы экспериментируете, Вы должны выбрать скорость обучения и другие гиперпараметры. При этом есть несколько важных моментов, которые нужно иметь в виду:

- Если параметры работают хорошо, вы должны видеть улучшение в течение нескольких сотен итераций;
- Помните о грубой и тонкой настройке гиперпараметров: начните с проверки  гиперпараметров в широком диапазоне   всего на нескольких обучающих итерациях, чтобы найти комбинации параметров, которые вообще работают;
- После того, как вы найдете несколько наборов параметров, которые работают,  найдите их более точные значения. Возможно, вам придется проводить обучение при большом числе эпох.
- Вы должны использовать валидационное множество для поиска гиперпараметров.


### Стремимся к лучшему
Если Вы вдохновились, то имеются много других функций, которые можно реализовать, чтобы попытаться повысить эффективность. Вы **не обязаны** их реализовывать, но не упустите шанс, если у вас есть время!

- Альтернативные оптимизаторы: вы можете попробовать Адам, Адаград, RMSprop и т. д.;
- Альтернативные функции активации, такие как  ReLU с утечкой, параметрическое ReLU, ELU или MaxOut;
- Ансамбли моделей;
- Расширение набора данных;
- Новые архитектуры.

  - [ResNets](https://arxiv.org/abs/1512.03385) , где вход  предыдущего слоя добавляется к выходу.
  - [DenseNets](https://arxiv.org/abs/1608.06993) , где входы предыдущих слоев объединяются вместе.
  - [Этот блог содержит подробный обзор](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)
  
### Успешного обучения! 


In [28]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        
        ############################################################################
        # ЗАДАНИЕ: Постройте модель, которая хорошо работает на CIFAR-10           #
        ############################################################################
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), strides=1, padding='same',
                                           activation=tf.nn.relu, kernel_initializer=initializer)
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2))
        self.norm1 = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.conv2 = tf.keras.layers.Conv2D(128, (3, 3), strides=1, padding='same',
                                           activation=tf.nn.relu, kernel_initializer=initializer)
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2))
        self.norm2 = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), strides=1, padding='same',
                                           activation=tf.nn.relu, kernel_initializer=initializer)
        self.pool3 = tf.keras.layers.MaxPool2D((2, 2))
        self.norm3 = tf.keras.layers.BatchNormalization(axis=-1)
        
        self.fc1 = tf.keras.layers.Dense(128, activation=tf.nn.relu, kernel_initializer=initializer)
        self.normfc1 = tf.keras.layers.BatchNormalization(axis=1)
        
        self.final = tf.keras.layers.Dense(10, activation='softmax', kernel_initializer=initializer)

        # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        ############################################################################
        #                           КОНЕЦ ВАШЕГО КОДА                              #
        ############################################################################
        
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # ЗАДАНИЕ: Постройте модель, которая хорошо работает на CIFAR-10           #
        ############################################################################
        # *****НАЧАЛО ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        
        x = self.conv1(input_tensor)
        x = self.norm1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.pool3(x)
        
        x = tf.keras.layers.Flatten()(x)
        x = self.fc1(x)
        x = self.normfc1(x)
        
        scores = self.final(x)

        # *****КОНЕЦ ВАШЕГО КОДА (НЕ УДАЛЯЙТЕ/НЕ МОДИФИЦИРУЙТЕ ЭТУ СТРОКУ)*****
        ############################################################################
        #                           КОНЕЦ ВАШЕГО КОДА                              #
        ############################################################################
                
        return scores

    
#device = '/device:GPU:0'   # выберите эту строку, если хотите использовать GPU!
device = '/cpu:0'           
print_every = 700
num_epochs = 15

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

# обучение на CPU на одной эпохе займет время !
train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Итерация 0, Эпоха 1, Потери: 3.3723158836364746, Точность: 10.9375, Val Потери: 4.072357177734375, Val Точность: 12.700000762939453
Итерация 700, Эпоха 1, Потери: 1.1808515787124634, Точность: 58.603782653808594, Val Потери: 0.9286485910415649, Val Точность: 68.80000305175781
Итерация 1400, Эпоха 2, Потери: 0.7731051445007324, Точность: 72.99458312988281, Val Потери: 0.8115264177322388, Val Точность: 73.29999542236328
Итерация 2100, Эпоха 3, Потери: 0.579459011554718, Точность: 79.87149047851562, Val Потери: 0.804970383644104, Val Точность: 71.9000015258789
Итерация 2800, Эпоха 4, Потери: 0.431369811296463, Точность: 85.06150817871094, Val Потери: 0.8356218338012695, Val Точность: 74.29999542236328
Итерация 3500, Эпоха 5, Потери: 0.30733248591423035, Точность: 89.49156188964844, Val Потери: 0.8603623509407043, Val Точность: 73.5999984741211
Итерация 4200, Эпоха 6, Потери: 0.22446732223033905, Точность: 92.26751708984375, Val Потери: 0.9268920421600342, Val Точность: 74.5
Итерация 4900,

## Опишите Ваши эксперименты
В приведенной ниже ячейке Вы должны объяснить, что Вы делали. Также опишите какие-либо дополнительные особенности, которые Вы использовали, и / или любые графики, которые Вы получили в процессе обучения и тестирования сети.

ЗАДАНИЕ: Опишите, что Вы делали

Был написан код для модели свёрточной нейронной сети типа CustomConvNet. Модель классифицирует изображения 10 классов из датасета CIFAR-10.

Модель состоит из трёх сверточных слоев. Использован фильтр 3x3, количество фильтров: 64, 128, 128, пулинг MaxPooling размера (2, 2), блочная нормализация (BatchNormalization), глобальный усредняющий пулинг. Регуляризация не используется.

Модель классификатора достигает поставленной точности выше 70% послей 2 эпохи обучения, всего было пройдено 15 эпох с финальной точностью 76%. Полученный результат сочтён удовлетворительным.